In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-ml").getOrCreate()

In [4]:
training = spark.read.csv("dataset/test1.csv", header=True, inferSchema=True)

In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [12]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [13]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [14]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [15]:
output = featureassembler.transform(training)

In [16]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [17]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [20]:
finalized_data=output.select("Independent Features", "Salary")

In [21]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor=LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor=regressor.fit(train_data)

In [25]:
# Coefficients
regressor.coefficients

DenseVector([-383.9733, 1711.1853])

In [26]:
# Intercepts
regressor.intercept

23998.330550919385

In [27]:
## Prediction
pred_results = regressor.evaluate(test_data)

In [28]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 17646.07679465793|
|          [23.0,2.0]| 18000|18589.315525876566|
+--------------------+------+------------------+



In [30]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1617.6961602672473, 3674507.5961331767)